# Measuremments with Representative

### Imports

In [2]:
import os, sys
lib_path = os.path.abspath(os.path.join('..', '..', 'utils'))
sys.path.append(lib_path)
from GraphType import GraphStat
from GraphType import GraphCollection
from scipy import stats
from ipywidgets import interact, fixed, interactive
import readCSV as reader
import ipywidgets as widgets
import matplotlib.pyplot as plt
import random
import numpy as np
import constants


### Classes

* Record the distances of different metrics using a representative

In [3]:
class GraphDistanceWithRep:
    #init with a graph stat and a collection of graph stats
    def __init__(self, graphStat, rep):
        self.graph = graphStat
        self.rep = rep
        self.out_d_distance, _ = stats.ks_2samp(graphStat.out_d, rep.out_d)
        self.na_distance,_ = stats.ks_2samp(graphStat.na, rep.na)
        self.mpc_distance,_ = stats.ks_2samp(graphStat.mpc, rep.mpc)


### Methods


* Find the median ks distance of the same number of nodes

In [4]:
def find_median(x, metric_distances):
    distance_dic = {}
    for index, num_of_nodes in enumerate(x):
        if num_of_nodes[0] not in distance_dic:
            distance_dic[num_of_nodes[0]] = []
        distance_dic[num_of_nodes[0]].append(metric_distances[index])
    median_x = []
    y = []
    for num_of_nodes, distances in distance_dic.items():
        median_x.append(num_of_nodes)
        y.append(np.median(distances))
    order = np.argsort(median_x)
    median_x = np.array(median_x)[order]
    median_y = np.array(y)[order]
    return median_x, median_y


* Plot Diagram

In [38]:
# metric_selector: GraphDistance -> float
def plot(infos, lines, id, metric_selector,colors, title, foldername):
    metric_distances = retrive_info_from_list(metric_selector, list(infos.values()))
    x = retrive_info_from_list(lambda a : a.graph.num_nodes, list(infos.values()))
    graph = plt.figure(id,figsize=(18, 10))
    plt.title(title)
    plt.plot(x, metric_distances, color='red', linestyle='', marker='o',alpha=0.7)
    #plot ks distance median
    median_x, median_y = find_median(x, metric_distances)
    plt.plot(median_x, median_y, color='black',marker='o')
    for i in range(0, len(lines)):
        line_infos = retrive_info_from_list(lambda a: infos[a], lines[i])
        line_y = retrive_info_from_list(metric_selector, line_infos)
        line_x = retrive_info_from_list(lambda a : a.graph.num_nodes, line_infos)
        plt.plot(line_x, line_y, marker='o', color=colors[i])
    mkdir_p(foldername)
    plt.savefig(fname = foldername+title+'.jpg', dpi=150)
    #graph.show()

* Retrieve information from a list 

In [6]:
def retrive_info_from_list(selector, distances):
    return list(map(selector, distances))

In [7]:
def readStats(path, numModels):
    names =  reader.readmultiplefiles(path, numModels, False)
    stats = []
    for name in names:
        stats.append(GraphStat(name))
    return stats

In [8]:
def calDistanceDic(stats, rep):
    dic = {}
    for info in stats:
        info = GraphDistanceWithRep(info, rep)
        dic[info.graph.id] = info
    return dic

In [25]:
def createRandomColors(size):
    #generate random color for each line
    colors = []

    for i in range(0, size):
        color = "#%06x" % random.randint(0, 0xFFFFFF)
        colors.append(color)
    return colors

In [43]:
def createSelectionWidge(options):
    w = widgets.SelectMultiple(
        options = options,
        value = [],
        description='Trajectory:',
        disabled=False,
    )
    return w

In [33]:
def mkdir_p(mypath):
    '''Creates a directory. equivalent to using mkdir -p on the command line'''

    from errno import EEXIST
    from os import makedirs,path

    try:
        makedirs(mypath)
    except OSError as exc: # Python >2.5
        if exc.errno == EEXIST and path.isdir(mypath):
            pass
        else: raise

## Metrics During GenerationPlots

### Read Human Representatives

In [42]:
### Read Models
#read representative
human_rep = GraphStat(constants.HUMAN_OUT_D_REP)
human_na = GraphStat(constants.HUMAN_NA_REP)
human_mpc = GraphStat(constants.HUMAN_MPC_REP)

# assign rep distributions to human_rep
human_rep.na = human_na.na
human_rep.mpc = human_mpc.mpc

## Viatra No Constraint

In [15]:
# Read generated models
viatra_no_con_stats = readStats('../input/viatra_nocon_output/', 5000)
viatra_no_con_dic = calDistanceDic(viatra_no_con_stats, human_rep)

In [46]:
filenames = reader.readmultiplefiles('../input/viatra_nocon_output/trajectories/', 15, False)
trajectories = {}
for name in filenames:
    trajectories[name] = reader.readTrajectory(name)
w = createSelectionWidge(trajectories)
colors = createRandomColors(len(trajectories))

In [77]:
def plot_out_degree(lines):
    plot(viatra_no_con_dic, lines, 0, lambda a: a.out_d_distance, colors, 'out degree', '../output/viatra_no_constraints/')
interact(plot_out_degree, lines=w)

interactive(children=(SelectMultiple(description='Trajectory:', options={}, value=()), Output()), _dom_classes…

<function __main__.plot_out_degree(lines)>

In [78]:
def plot_out_na(lines):
    plot(viatra_no_con_dic, lines, 0, lambda a: a.na_distance, colors, 'node activity', '../output/viatra_no_constraints/')
interact(plot_out_na, lines=w)

interactive(children=(SelectMultiple(description='Trajectory:', options={}, value=()), Output()), _dom_classes…

<function __main__.plot_out_na(lines)>

In [79]:
def plot_out_mpc(lines):
    plot(viatra_no_con_dic, lines, 0, lambda a: a.mpc_distance, colors, 'MPC', '../output/viatra_no_constraints/')
interact(plot_out_mpc, lines=w)

interactive(children=(SelectMultiple(description='Trajectory:', options={}, value=()), Output()), _dom_classes…

<function __main__.plot_out_mpc(lines)>

## Viatra with constraints

In [50]:
viatra_con_stats = readStats('../input/viatra_con_output/',5000)
viatra_con_dic = calDistanceDic(viatra_con_stats, human_rep)

# trajectories and colors
trajectories = {}
w = createSelectionWidge(trajectories)
colors = createRandomColors(len(trajectories))

In [51]:
def plot_out_degree(lines):
    plot(viatra_con_dic, lines, 0, lambda a: a.out_d_distance, colors, 'out degree', '../output/viatra_constraints/')
interact(plot_out_degree, lines=[[]])

interactive(children=(Dropdown(description='lines', options=([],), value=[]), Output()), _dom_classes=('widget…

<function __main__.plot_out_degree(lines)>

In [52]:
def plot_na(lines):
    plot(viatra_con_dic, lines, 0, lambda a: a.na_distance, colors, 'node activity', '../output/viatra_constraints/')
interact(plot_na, lines=[[]])

interactive(children=(Dropdown(description='lines', options=([],), value=[]), Output()), _dom_classes=('widget…

<function __main__.plot_na(lines)>

In [53]:
def plot_mpc(lines):
    plot(viatra_con_dic, lines, 0, lambda a: a.mpc_distance, colors, 'MPC', '../output/viatra_constraints/')
interact(plot_mpc, lines=[[]])

interactive(children=(Dropdown(description='lines', options=([],), value=[]), Output()), _dom_classes=('widget…

<function __main__.plot_mpc(lines)>

## Controlled RandomEMF

In [59]:
random_emf_stats = readStats('../input/random_emf_output/',5000)
random_emf_dic = calDistanceDic(random_emf_stats, human_rep)

# trajectories and colors
trajectories = {}
w = createSelectionWidge(trajectories)
colors = createRandomColors(len(trajectories))

In [60]:
def plot_out_degree(lines):
    plot(random_emf_dic, lines, 0, lambda a: a.out_d_distance, colors, 'out degree', '../output/random_emf/')
interact(plot_out_degree, lines=[[]])

interactive(children=(Dropdown(description='lines', options=([],), value=[]), Output()), _dom_classes=('widget…

<function __main__.plot_out_degree(lines)>

In [61]:
def plot_node_activity(lines):
    plot(random_emf_dic, lines, 0, lambda a: a.na_distance, colors, 'node activity', '../output/random_emf/')
interact(plot_node_activity, lines=[[]])

interactive(children=(Dropdown(description='lines', options=([],), value=[]), Output()), _dom_classes=('widget…

<function __main__.plot_node_activity(lines)>

In [62]:
def plot_mpc(lines):
    plot(random_emf_dic, lines, 0, lambda a: a.mpc_distance, colors, 'mpc', '../output/random_emf/')
interact(plot_mpc, lines=[[]])

interactive(children=(Dropdown(description='lines', options=([],), value=[]), Output()), _dom_classes=('widget…

<function __main__.plot_mpc(lines)>

## Controlled Viatra with MPC

In [67]:
con_viatra_stats = readStats('../input/controled_viatra_mpc/',5000)
con_viatra_dic = calDistanceDic(con_viatra_stats, human_rep)

# trajectories and colors
trajectories = {}
w = createSelectionWidge(trajectories)
colors = createRandomColors(len(trajectories))

In [74]:
def plot_out_degree(lines):
    plot(con_viatra_dic, lines, 0, lambda a: a.out_d_distance, colors, 'out_degree', '../output/controled_viatra_with_mpc/')
interact(plot_out_degree, lines=w)

interactive(children=(SelectMultiple(description='Trajectory:', options={}, value=()), Output()), _dom_classes…

<function __main__.plot_out_degree(lines)>

In [75]:
def plot_na(lines):
    plot(con_viatra_dic, lines, 0, lambda a: a.na_distance, colors, 'Node Activity', '../output/controled_viatra_with_mpc/')
interact(plot_na, lines=w)

interactive(children=(SelectMultiple(description='Trajectory:', options={}, value=()), Output()), _dom_classes…

<function __main__.plot_na(lines)>

In [76]:
def plot_mpc(lines):
    plot(con_viatra_dic, lines, 0, lambda a: a.mpc_distance, colors, 'mpc', '../output/controled_viatra_with_mpc/')
interact(plot_mpc, lines=w)

interactive(children=(SelectMultiple(description='Trajectory:', options={}, value=()), Output()), _dom_classes…

<function __main__.plot_mpc(lines)>

## (Pseudo) Random EMF instantiator

In [80]:
random_emf_stats = readStats('../input/real_random_output/',5000)
random_emf_dic = calDistanceDic(random_emf_stats, human_rep)

# trajectories and colors
trajectories = {}
w = createSelectionWidge(trajectories)
colors = createRandomColors(len(trajectories))

In [82]:
def plot_out_degree(lines):
    plot(random_emf_dic, lines, 0, lambda a: a.out_d_distance, colors, 'out_degree', '../output/random_emf_instantiator/')
interact(plot_out_degree, lines=w)

interactive(children=(SelectMultiple(description='Trajectory:', options={}, value=()), Output()), _dom_classes…

<function __main__.plot_out_degree(lines)>

In [83]:
def plot_na(lines):
    plot(random_emf_dic, lines, 0, lambda a: a.na_distance, colors, 'Node Activity', '../output/random_emf_instantiator/')
interact(plot_na, lines=w)

interactive(children=(SelectMultiple(description='Trajectory:', options={}, value=()), Output()), _dom_classes…

<function __main__.plot_na(lines)>

In [84]:
def plot_mpc(lines):
    plot(random_emf_dic, lines, 0, lambda a: a.mpc_distance, colors, 'mpc', '../output/random_emf_instantiator/')
interact(plot_mpc, lines=w)

interactive(children=(SelectMultiple(description='Trajectory:', options={}, value=()), Output()), _dom_classes…

<function __main__.plot_mpc(lines)>